# Land Use

Another measure of urban form is land use.  I'll start with zoning information from City of LA planning department.  Ultimately we'll want to use parcel data from LA County.

Much as I did with streets I'll focus on a couple of NCs while I try to figure this out.

https://data.lacity.org/browse?q=zoning&sortBy=relevance

https://planning.lacounty.gov/luz


In [ ]:
import nc

KG = nc.KG()

In [ ]:
ecwa = KG.nc_poly(79)

In [ ]:
encino = KG.nc_poly(18)

This is the City of LA zoning file referenced above.

In [ ]:
%time
#zone_gdf = gpd.read_file('/home/mcmorgan/SOCALKG/data/LA_city/zoning/Zoning.zip')

zone_gdf = gpd.read_file('https://data.lacity.org/api/geospatial/rryw-49uv?method=export&format=Shapefile')

In [ ]:
len(zone_gdf)

Reasonable size for browser-based viz.  Use the NC poly's to subset the zone info.

**Note to self:** - KG class is deficient.  You need to revisit!

In [ ]:
ecwa_gdf = KG.nc_gfd

In [ ]:
ecwa_gdf

Ok.  We have a one row gdf for Encino.  Spatial join hacking to use this poly to select zoning poly's.

In [ ]:
ecwa_zone_gdf = zone_gdf.to_crs("EPSG:4326").sjoin(ecwa_gdf, how="inner", predicate="within")

In [ ]:
len(ecwa_zone_gdf)

In [ ]:
ecwa_zone_gdf[['shape_area', 'zone_cmplt', 'NAME', 'geometry']].explore(cmap="Blues")

geopandas explore function uses folium.  That is sufficient to get a sense for what we have.

In [ ]:
ecwa_zone_gdf.info()

Would like to get a better sense for the actual zone codes.  

**Note:** what is cmplt?

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
ecwa_zone_gdf.zone_cmplt.value_counts(sort=False)

Next step would be to build some logic to group the code types.  I'm really looking for something like residential, commercial, open space.

Once you have the groups, a simple sum should get the area, i.e. like this.  [This](https://planning.lacity.org/odocument/eadcb225-a16b-4ce6-bc94-c915408c2b04/Zoning_Code_Summary.pdf) will help.

**Note:** what are the units for the area?  Should I just use shapely and compute them myself?

In [ ]:
ecwa_zone_gdf.query(f"zone_cmplt == 'R1-1-RIO'")['shape_area'].sum()

**Note:** As with streets, need a utility to compute density.  

In [ ]:
zone_ref = pd.read_csv('/home/mcmorgan/SOCALKG/data/LA_city/zoning/Zoning_Reference_Table.csv')

In [ ]:
pd.set_option("display.max_rows", 10)

In [ ]:
zone_ref

In [ ]:
zone_ref.ZONE_CLASS.value_counts().to_frame().head(50)

In [ ]:
neighborhoods_gdf = gpd.read_file('../data/neighborhoods/Neighborhood_Councils_(Certified)_cleaned.shp')
nc_layer = GeoData(geo_dataframe = neighborhoods_gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.8, 'weight':1.9, 'dashArray':'5', 'fillOpacity':0.2},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Neighborhood Council')
